In [6]:
import logging
from mecom import MeCom, ResponseException, WrongChecksum
from serial import SerialException
import time
from threading import Thread

In [7]:
import ipywidgets as widgets
from IPython.display import display

In [8]:
from mecom.commands import PARAMETERS

## Open device

In [9]:
dev = MeCom("/dev/ttyUSB0")
address = dev.identify()

In [10]:
queries = [i for i in PARAMETERS if i["id"]<2000 and i["id"]>=1000]
status, i_lim, u_lim, set_T = [i for i in PARAMETERS if i["id"]==3000 or i["id"]==2010 or i["id"]==2030 or i["id"]==2031]

In [11]:
queries

[{'format': 'FLOAT32', 'id': 1000, 'name': 'Object Temperature'},
 {'format': 'FLOAT32', 'id': 1001, 'name': 'Sink Temperature'},
 {'format': 'FLOAT32', 'id': 1010, 'name': 'Target Object Temperature'},
 {'format': 'FLOAT32', 'id': 1011, 'name': 'Ramp Object Temperature'},
 {'format': 'FLOAT32', 'id': 1020, 'name': 'Actual Output Current'},
 {'format': 'FLOAT32', 'id': 1021, 'name': 'Actual Output Voltage'},
 {'format': 'INT32', 'id': 1046, 'name': 'Object Sensor Type'},
 {'format': 'INT32', 'id': 1200, 'name': 'Temperature is Stable'}]

In [12]:
query_widgets = []
style = {'description_width': 'initial'}
for i in queries:
    query_widgets.append(widgets.FloatText(description=i["name"], disabled=True, style=style))
readout = widgets.VBox(query_widgets)

enable = widgets.ToggleButton(
                             value=False,
                             description='Start',
                             disabled=False,
                             button_style='', # 'success', 'info', 'warning', 'danger' or ''
                             tooltip='Enable T Control',
                             icon='check'
                             )

setpoint = widgets.BoundedFloatText(value=20, min=10, max=35, step=0.1,
                                    description="T setpoint", continuous_update=False)

i_lim_set = widgets.BoundedFloatText(value=1.0, min=0.5, max=1.5, step=0.1,
                                     description="Current Limit", continuous_update=False)
u_lim_set = widgets.BoundedFloatText(value=8.5, min=5.0, max=9.0, step=0.1,
                                     description="Voltage Limit", continuous_update=False)

confirm = widgets.Button(description='Set',
                         disabled=False,
                         button_style='', # 'success', 'info', 'warning', 'danger' or ''
                         tooltip='Send parameters to TEC',
                         icon='check')

settings = widgets.HBox([widgets.VBox([setpoint, i_lim_set, u_lim_set]), confirm])

In [13]:
settings_dict = {set_T["id"]: setpoint, i_lim["id"]: i_lim_set, u_lim["id"]: u_lim_set}

In [14]:
display(readout)
display(settings)
display(enable)

ToggleButton(value=False, description='Start', icon='check', tooltip='Enable T Control')

In [15]:
def get_temp():
    confirm.on_click(on_confirm)
    while getattr(t_t, "do_run", True):
        if getattr(t_t, "cap", True):
            try:
                for idx,i in enumerate(queries):
                    query_widgets[idx].value = dev.get_parameter(parameter_id=i["id"], address=address, parameter_instance=1)
            except:
                time.sleep(0.1)
        else:
            time.sleep(0.1)
            
def on_confirm(new_val):
    if not enable.value:
        time.sleep(0.1)
        t_t.cap = False
        for i in [set_T, i_lim, u_lim]:
            time.sleep(0.1)
            dev.set_parameter(parameter_id=i["id"], 
                              value=settings_dict[i["id"]].value, 
                              address=address, 
                              parameter_instance=1)
        t_t.cap = True
    else:
        print("Can't set during operation.")
        
def on_set(change):
    t_t.cap = False
    time.sleep(0.1)
    setter = int(change["new"])
    dev.set_parameter(parameter_id=status["id"], 
                     value=setter, 
                     address=address, 
                     parameter_instance=1)
    t_t.cap = True

t_t = Thread(target=get_temp)
t_t.daemon = True
t_t.cap = True
t_t.start()

enable.observe(on_set, names='value')

In [16]:
t_t.do_run = False
t_t.join()
dev.stop()